# Artificial Intelligence

## Convolutional Neural Networks

## Project: Write an Algorithm for a Crop Disease Classification App

---




---

### The Road Ahead

We break the notebook into separate steps.  Feel free to use the links below to navigate the notebook.


---
<a id='step0'></a>
## Step 0: Create the datasets (train,test,valid from the original dataset)

### download the original dataset
download the initial dataset from this url :https://zenodo.org/record/1204914/files/plantvillage_deeplearning_paper_dataset.7z
In the code cell below, we import a dataset of crop images.  We populate a few variables through the use of the `load_files` function from the scikit-learn library:
- `train_files`, `valid_files`, `test_files` - numpy arrays containing file paths to images
- `train_targets`, `valid_targets`, `test_targets` - numpy arrays containing onehot-encoded classification labels 
- `crop_names` - list of string-valued crop names for translating labels

## Step 1 : Renaming the file in the original dataSet

In [ ]:
# rename the files in all_data_set so they allways have as prefix the directory name related
import os
import re
from pathlib import Path

for f in os.listdir(all_data_dir):
  foldername = f
  for file in os.listdir( all_data_dir + '/' + f):
    oldfile= Path(all_data_dir + '/' + f + '/' + file)
    if len(file)>50:
      index = file.rfind('_')
      
      #print (file[index:]) 
      #print (foldername + file[index:])
      newfile = Path(all_data_dir + '/' + f + '/' + foldername + file[index:])
      print (newfile)
      p=oldfile
      p.rename(newfile)

## Step 2 : Creation of the Train/Valid/Test DataSet

In [36]:
def split_dataset_into_test_and_train_sets(all_data_dir, training_data_dir, testing_data_dir, testing_data_pct):
    # Recreate testing and training directories
    if testing_data_dir.count('/') > 1:
        shutil.rmtree(testing_data_dir, ignore_errors=False)
        os.makedirs(testing_data_dir)
        print("Successfully cleaned directory " + testing_data_dir)
    else:
        print("Refusing to delete testing data directory " + testing_data_dir + " as we prevent you from doing stupid things!")

    if training_data_dir.count('/') > 1:
        shutil.rmtree(training_data_dir, ignore_errors=False)
        os.makedirs(training_data_dir)
        print("Successfully cleaned directory " + training_data_dir)
    else:
        print("Refusing to delete testing data directory " + training_data_dir + " as we prevent you from doing stupid things!")

    num_training_files = 0
    num_testing_files = 0

    for subdir, dirs, files in os.walk(all_data_dir):
        category_name = os.path.basename(subdir)

        # Don't create a subdirectory for the root directory
        print(category_name + " vs " + os.path.basename(all_data_dir))
        if category_name == os.path.basename(all_data_dir):
            continue

        training_data_category_dir = training_data_dir + '/' + category_name
        testing_data_category_dir = testing_data_dir + '/' + category_name

        if not os.path.exists(training_data_category_dir):
            os.mkdir(training_data_category_dir)

        if not os.path.exists(testing_data_category_dir):
            os.mkdir(testing_data_category_dir)

        for file in files:
            input_file = os.path.join(subdir, file)
            if np.random.rand(1) < testing_data_pct:
                shutil.copy(input_file, testing_data_dir + '/' + category_name + '/' + file)
                num_testing_files += 1
            else:
                shutil.copy(input_file, training_data_dir + '/' + category_name + '/' + file)
                num_training_files += 1

    print("Processed " + str(num_training_files) + " training files.")
    print("Processed " + str(num_testing_files) + " testing files.")

In [1]:
import os
import shutil, sys
import numpy as np
all_data_dir='../DataSet/alldataset'
training_data_dir='../DataSet/traindataset'
training_data_dir2='../DataSet/train'
testing_data_dir='../DataSet/test'
valid_data_dir='../DataSet/valid'

#split_dataset_into_test_and_train_sets(all_data_dir, training_data_dir, testing_data_dir, 0.3)
#split_dataset_into_test_and_train_sets(training_data_dir, training_data_dir2, valid_data_dir, 0.3)    

## Step 3 : Loading of the DataSets

In [2]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    crop_files = np.array(data['filenames'])
    crop_targets = np_utils.to_categorical(np.array(data['target']), 39)
    return crop_files, crop_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('../DataSet/train')
valid_files, valid_targets = load_dataset('../DataSet/valid')
test_files, test_targets = load_dataset('../DataSet/test')

print (len(train_targets))

# load list of crop names
crop_names = [item[17:-1] for item in sorted(glob("../DataSet/train/*/"))]
#for item in sorted(glob("../DataSet/train/*/")):
# print statistics about the dataset
print('There are %d total crop categories.' % len(crop_names))
# print('There are %s total dog images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training crop images.' % len(train_files))
print('There are %d validation crop images.' % len(valid_files))
print('There are %d test crop images.'% len(test_files))

/usr/lib64/python3.4/importlib/_bootstrap.py:321: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  return f(*args, **kwds)
Using TensorFlow backend.


18844
There are 39 total crop categories.
There are 18844 training crop images.
There are 8063 validation crop images.
There are 11664 test crop images.


## Step 4 : Pre-process the Data

When using TensorFlow as backend, Keras CNNs require a 4D array (which we'll also refer to as a 4D tensor) as input, with shape

$$
(\text{nb_samples}, \text{rows}, \text{columns}, \text{channels}),
$$

where `nb_samples` corresponds to the total number of images (or samples), and `rows`, `columns`, and `channels` correspond to the number of rows, columns, and channels for each image, respectively.  

The `path_to_tensor` function below takes a string-valued file path to a color image as input and returns a 4D tensor suitable for supplying to a Keras CNN.  The function first loads the image and resizes it to a square image that is $224 \times 224$ pixels.  Next, the image is converted to an array, which is then resized to a 4D tensor.  In this case, since we are working with color images, each image has three channels.  Likewise, since we are processing a single image (or sample), the returned tensor will always have shape

$$
(1, 224, 224, 3).
$$

The `paths_to_tensor` function takes a numpy array of string-valued image paths as input and returns a 4D tensor with shape 

$$
(\text{nb_samples}, 224, 224, 3).
$$

Here, `nb_samples` is the number of samples, or number of images, in the supplied array of image paths.  It is best to think of `nb_samples` as the number of 3D tensors (where each 3D tensor corresponds to a different image) in your dataset!

In [3]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [4]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 



# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

100%|██████████| 11664/11664 [00:18<00:00, 630.50it/s]


### (IMPLEMENTATION) Train the Model

Train your model in the code cell below.  Use model checkpointing to save the model that attains the best validation loss.

You are welcome to [augment the training data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html), but this is not a requirement. 

---
<a id='step4'></a>
## Step 5: Use the VGG16 (Transfer Learning)

To reduce training time without sacrificing accuracy, we show you how to train a CNN using transfer learning.  In the following step, you will get a chance to use transfer learning to train your own CNN.

### Obtain Bottleneck Features

In [5]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt
import math
import cv2

# dimensions of our images.
img_width, img_height = 224, 224

train_data_dir = training_data_dir2
validation_data_dir = valid_data_dir
test_data_dir = testing_data_dir

# number of epochs to train top model
epochs = 50
# batch size used by flow_from_directory and predict_generator
batch_size = 16


def save_bottlebeck_features():
    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    datagen = ImageDataGenerator(rescale=1. / 255)

    if False:
      generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)

      print(len(generator.filenames))
      print(generator.class_indices)
      print(len(generator.class_indices))

      nb_train_samples = len(generator.filenames)
      num_classes = len(generator.class_indices)

      predict_size_train = int(math.ceil(nb_train_samples / batch_size))

      bottleneck_features_train = model.predict_generator(
        generator, predict_size_train)

      np.save('bottleneck_features_train.npy', bottleneck_features_train)

      generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)

      nb_validation_samples = len(generator.filenames)

      predict_size_validation = int(
        math.ceil(nb_validation_samples / batch_size))

      bottleneck_features_validation = model.predict_generator(
        generator, predict_size_validation)

      np.save('bottleneck_features_validation.npy',
            bottleneck_features_validation)

      generator = datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)

      nb_test_samples = len(generator.filenames)

      predict_size_test = int(
        math.ceil(nb_test_samples / batch_size))

      bottleneck_features_test = model.predict_generator(
        generator, predict_size_test)

      np.save('bottleneck_features_test.npy',
            bottleneck_features_test)
    
    
    
save_bottlebeck_features()

In [8]:
'''
Using Bottleneck Features for Multi-Class Classification in Keras
We use this technique to build powerful (high accuracy without overfitting) Image Classification systems with small
amount of training data.
'''
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt
import math
import cv2


# dimensions of our images.
img_width, img_height = 224, 224

top_model_weights_path = 'saved_models/weights.best.vgg16.hdf5'
train_data_dir = training_data_dir2
validation_data_dir = valid_data_dir
test_data_dir = testing_data_dir


In [9]:
train_VGG16 = np.load('bottleneck_features_train.npy')
valid_VGG16 = np.load('bottleneck_features_validation.npy')
test_VGG16 =  np.load('bottleneck_features_test.npy')


print(len(train_VGG16))
print(len(valid_VGG16))
print(len(test_VGG16))

18844
8063
11664


### Model Architecture

The model uses the the pre-trained VGG16 model as a fixed feature extractor, where the last convolutional output of VGG16 is fed as input to our model.  We only add a global average pooling layer and a fully connected layer, where the latter contains one node for each crop category and is equipped with a softmax.

In [33]:
from keras.callbacks import ModelCheckpoint
# number of epochs to train top model
epochs = 10
# batch size used by flow_from_directory and predict_generator
batch_size = 16

#datagen_top = ImageDataGenerator(rescale=1. / 255)
#generator_top = datagen_top.flow_from_directory(
#    train_data_dir,
#    target_size=(img_width, img_height),
#    batch_size=batch_size,
#    class_mode='categorical',
#    shuffle=False)

#nb_train_samples = len(generator_top.filenames)
#num_classes = len(generator_top.class_indices)
# save the class indices to use use later in predictions
#np.save('class_indices.npy', generator_top.class_indices)

#generator_top = datagen_top.flow_from_directory(
#    validation_data_dir,
#    target_size=(img_width, img_height),
#    batch_size=batch_size,
#    class_mode=None,
#    shuffle=False)

#nb_validation_samples = len(generator_top.filenames)

model = Sequential()
model.add(Flatten(input_shape=train_VGG16.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(39, activation='softmax'))
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               6422784   
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 39)                10023     
Total params: 6,432,807
Trainable params: 6,432,807
Non-trainable params: 0
_________________________________________________________________


In [ ]:

model.compile(optimizer='RMSprop',loss='categorical_crossentropy', metrics=['accuracy'])
#model.compile(optimizer="RMSprop", loss="mean_squared_error", metrics=["accuracy"] )
#from keras.optimizers import SGD
#model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

checkpointer = ModelCheckpoint(filepath=top_model_weights_path, verbose=1, save_best_only=True)

history=model.fit(train_VGG16, train_targets,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(valid_VGG16, valid_targets),callbacks=[checkpointer], verbose=1)

model.save('crop_model.h5')


### Finetune the model

In [ ]:
top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

# add the model on top of the convolutional base
model.add(top_model)
We then proceed to freeze all convolutional layers up to the last convolutional block:

# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])
Finally, we start training the whole thing, with a very slow learning rate:

batch_size = 16

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary')

# fine-tune the model
model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size)

In [ ]:
### TODO: Load the model weights with the best validation loss.
model.load_weights(top_model_weights_path)
VGG16_predictions = [np.argmax(model.predict(np.expand_dims(feature, axis=0))) for feature in test_VGG16]

# report test accuracy
test_accuracy = 100*np.sum(np.array(VGG16_predictions)==np.argmax(test_targets, axis=1))/len(VGG16_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

## TODO: Visualize the training and validation loss of your neural network
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()



(eval_loss, eval_accuracy) = model.evaluate(
    valid_VGG16, valid_targets, batch_size=batch_size, verbose=1)

print("[INFO] accuracy: {:.2f}%".format(eval_accuracy * 100))
print("[INFO] Loss: {}".format(eval_loss))

plt.figure(1)

# summarize history for accuracy

plt.subplot(211)
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

# summarize history for loss

plt.subplot(212)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


## BUILD MODEL FROM SCRATCH

In [24]:
# Import deep learning resources from Keras
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Dropout
from keras.layers import Flatten, Dense
from keras.layers import Conv2D, GlobalAveragePooling2D

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential


### TODO: Define your architecture.

#def create_model():
# create model
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu', 
                        input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=128, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=256, kernel_size=2, kernel_initializer='he_normal', activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(GlobalAveragePooling2D())

model.add(Dense(39, activation='softmax'))

model.summary()



model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 224, 224, 16)      208       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 112, 112, 16)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 112, 112, 32)      2080      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 56, 56, 64)        8256      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 28, 28, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 28, 28, 128)       32896     
__________

In [27]:
from keras.callbacks import ModelCheckpoint  

### TODO: specify the number of epochs that you would like to use to train the model.

epochs = 10

### Do NOT modify the code below this line.

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=1)

Train on 18844 samples, validate on 8063 samples
Epoch 1/10
18840/18844 [============================>.] - ETA: 0s - loss: 0.0826 - acc: 0.9768
Epoch 00001: val_loss improved from inf to 0.08007, saving model to saved_models/weights.best.from_scratch.hdf5
18844/18844 [==============================] - 67s 4ms/step - loss: 0.0826 - acc: 0.9768 - val_loss: 0.0801 - val_acc: 0.9770
Epoch 2/10
18840/18844 [============================>.] - ETA: 0s - loss: 0.0495 - acc: 0.9835
Epoch 00002: val_loss improved from 0.08007 to 0.03878, saving model to saved_models/weights.best.from_scratch.hdf5
18844/18844 [==============================] - 66s 3ms/step - loss: 0.0495 - acc: 0.9835 - val_loss: 0.0388 - val_acc: 0.9869
Epoch 3/10
18840/18844 [============================>.] - ETA: 0s - loss: 0.0355 - acc: 0.9877
Epoch 00003: val_loss did not improve
18844/18844 [==============================] - 66s 3ms/step - loss: 0.0355 - acc: 0.9877 - val_loss: 0.0727 - val_acc: 0.9771
Epoch 4/10
18840/18844

## try with augmented data

In [26]:
batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(224, 224),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        valid_data_dir,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='binary')

model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)
model.save_weights(top_model_weights_path)  # always save your weights after training or during training

Found 18844 images belonging to 39 classes.
Found 8063 images belonging to 39 classes.
Epoch 1/10


ValueError: Error when checking target: expected dense_8 to have shape (39,) but got array with shape (1,)

In [28]:
model.load_weights('saved_models/weights.best.from_scratch.hdf5')

# get index of predicted crop for each image in test set
crop_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(crop_predictions)==np.argmax(test_targets, axis=1))/len(crop_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 88.7346%


### Predict with the model from scratch

In [31]:

def predict_breed_fromscratch(img_path):
    # obtain predicted vector
    imgtensor=path_to_tensor(img_path)
    predicted_vector = model.predict(imgtensor)
    return crop_names[np.argmax(predicted_vector)]

In [32]:
import random
random.seed(8675309)
from keras.preprocessing import image
various_files = np.array(glob("images/*"))
random.shuffle(various_files)

from keras.preprocessing import image
for image_path in various_files:
    try:
        print(image_path)  
        cropname=predict_breed_fromscratch(image_path)
        print("crop:{}".format(cropname))
        
    except:
        break

images/tomatoseptoria2.JPG
crop:Apple_Frogeye_Spot
images/tomato-yellow-leaf-curl.JPG
crop:Pepper_bell___healthy
images/peachhealthy.JPG
crop:Blueberry___healthy
images/applescab.JPG
crop:Pepper_bell___healthy
images/blueberryhealthy.JPG
crop:Blueberry___healthy
images/tomatoseptoria.JPG
crop:Tomato___Septoria_leaf_spot


### Predict  with the VGG16

In [19]:
from extract_bottleneck_features import *

def VGG16_predict_breed(img_path):
    # extract bottleneck features
    bottleneck_feature = extract_VGG16(path_to_tensor(img_path))
    # obtain predicted vector
    predicted_vector = model.predict(bottleneck_feature)
    return crop_names[np.argmax(predicted_vector)]

In [21]:
import random
random.seed(8675309)
from keras.preprocessing import image
various_files = np.array(glob("images/*"))
random.shuffle(various_files)

from keras.preprocessing import image
for image_path in various_files:
    try:
        print(image_path)  
        cropname=VGG16_predict_breed(image_path)
        print("crop:{}".format(cropname))
        
    except:
        break

images/tomatoseptoria2.JPG
crop:Apple_Frogeye_Spot
images/tomato-yellow-leaf-curl.JPG
crop:Apple_Frogeye_Spot
images/peachhealthy.JPG
crop:Apple_Frogeye_Spot
images/applescab.JPG
crop:Apple_Frogeye_Spot
images/blueberryhealthy.JPG
crop:Apple_Frogeye_Spot
images/tomatoseptoria.JPG
crop:Apple_Frogeye_Spot


## Fine tuning of the model